## Initial Code Analysis ##

In [3]:
# setup steps #
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

ocha_data_df = pd.read_csv('ocha_weighted_avg.csv')
piv_ocha_df = pd.read_csv('ocha_needs_met_pivot.csv')

In [4]:
# group and pivot for OCHA needs met #
piv_ocha_df = piv_ocha_df.reset_index(drop=True) 
piv_ocha_df['pct_fund'] = piv_ocha_df['% funding']
piv_ocha_df['pct_fund'] = piv_ocha_df['pct_fund'].str.replace('%', '', regex=False)
piv_ocha_df['pct_fund'] = pd.to_numeric(piv_ocha_df['pct_fund'], errors= 'coerce')
piv_ocha_df['pct_fund'] = piv_ocha_df['pct_fund']/100

# exploding out the data from the grouped format
def split_and_explode_countries(df):
    df_new = df.copy()

    def handle_country(country):
        if isinstance(country, str):
            return country.split(',')
        elif isinstance(country, list):
            return country
        else: 
            try: 
                return [str(country)] 
            except Exception: 
                return [str(country)]  # Handle potential errors during conversion

    df_new['Country'] = df_new['Country'].apply(handle_country)
    df_new = df_new.explode('Country') 
    df_new['Country'] = df_new['Country'].str.strip() 

    return df_new

piv_ocha_df['Country'] = piv_ocha_df['Country'].astype(str)

new_pivdf = split_and_explode_countries(piv_ocha_df) 

# Group by country and year, calculate average % Funding
grouped_df = new_pivdf.groupby(['Country', 'Year'])['pct_fund'].mean().reset_index()

# filter to just our OCHA countries of interest
target_countries = ['Afghanistan', 'Burkina Faso', 'Cameroon', 'Central African Republic', 'Chad', 'Democratic Republic of the Congo', 'Haiti', 'Mali', 'Myanmar', 'Niger', 'Nigeria', 'Occupied Palestinian Territory', 'South Sudan', 'Somalia', 'Sudan', 'Syrian Arab Republic', 'Ukraine', 'Yemen']
filtered_df = grouped_df[grouped_df['Country'].isin(target_countries)]


# Create the line chart using Altair
chart = alt.Chart(filtered_df).mark_line().encode(
    x=alt.X('Year:O', title='Year'), 
    y=alt.Y('pct_fund:Q', axis=alt.Axis(format='%', title='Percentage Funding Request Recieved')), 
    color='Country',
    tooltip=['Year', 'Country', alt.Tooltip('pct_fund', format='.2%')]
).properties(
    title='Average % Funding by Country Over Time',
    width=600  
).interactive()

chart.show() 

alt.Chart(...)

# Adding variables, exploring our data #

In [ ]:
# Creating our Y-variable #

